https://huggingface.co/facebook/mbart-large-50-many-to-many-mmt

# Preparation

In [7]:
%pip install transformers sentencepiece pandas -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
!wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto

--2024-09-07 13:27:32--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14023 (14K) [text/plain]
Saving to: ‘sentencepiece_model.proto’

     0K .......... ...                                        100% 91.6M=0s

2024-09-07 13:27:32 (91.6 MB/s) - ‘sentencepiece_model.proto’ saved [14023/14023]



In [5]:
# ! protoc --python_out=. sentencepiece_model.proto

In [1]:
import pandas as pd
data = pd.read_csv('overall_80K.csv')
data.drop(columns=["Unnamed: 0"], inplace=True)
data

,target,source
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...
81141,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


# Counting words

In [2]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
print(tokenizer.vocab_size)

250054


In [4]:
all_sentences = data['target'].values
print(len(all_sentences))

81146


In [5]:
all_sentences[:10]

array(['Та пыгрисит маим вармаль э̄рнэ поратэт ат вермгыг варункв',
       'Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс. ',
       'Вит са̄мыл сунсым о̄нтыс ',
       'Атаявев, акваг лылынг тагл ворн та тотавев.',
       'Ман ты пӣлтал,  веськат хумиюв  нэтхуньт ат ёрувлвылув',
       'То̄лыг я̄ ко̄тильт та̄гапас', 'Лё̄ӈх ӯлтта тисупа тыламлас. ',
       'Салыл султтанэтэ, хунтлы, аманхотют савынкан торыг тортал та лющнэтэ суйты.',
       'Тыи сака ёмас сахи, щар я̄ӈк тӯйт хурипа.',
       'Нох-хилумтэ - тахурип ёмас аги. '], dtype=object)

In [6]:
from collections import Counter, defaultdict
from tqdm.auto import tqdm, trange
import random
import re

In [7]:
char_count = Counter()
for text in tqdm(all_sentences):
    char_count.update(text)

100%|██████████| 81146/81146 [00:00<00:00, 185867.35it/s]


In [8]:
PUNCT = '.,-—:)(»«!?–/;„"“…*́№Ёҥ[]”^%+І=і•_􏰀²|}{#‘■>⁠’á<°\§\''
SPACE = '▁'

In [9]:
for k, v in char_count.most_common(200):
    if not re.match('[а-яА-Яa-zA-Z0-9ё\']', k):
        print(k, end='')

 ̄.,ӈӯ-ēā:"ӣ«»ō?!–ЁӮĀ)(…	;Ӣ­_—Ō%Ē¬№][/\ūӑӇ=ӆ$”ә +ўѐӊ@’¸є#™<>

In [10]:
def get_words(text, tokenizer, verbose=False):
    toks = tokenizer.tokenize(text)
    words = []
    word = []
    prev_punct = False
    for tok in toks:
        is_punct = tok.lstrip(SPACE) and all(c in PUNCT for c in tok.lstrip(SPACE))
        if tok.startswith(SPACE) or prev_punct != is_punct:
            if word:
                words.append(word)
            word = []
        word.append(tok)
        prev_punct = is_punct
    if word:
        words.append(word)
    if verbose:
        print(words)
    res = words
    # assert tokenizer.decode([tok for t in res for tok in t]) == text
    return res

In [11]:
word_count = Counter()
word2toks = {}
for text in tqdm(all_sentences):
    for word_toks in get_words(text, tokenizer):
        word = ''.join(word_toks)
        word_count[word] += 1
        word2toks[word] = word_toks
print(len(word_count))

100%|██████████| 81146/81146 [00:07<00:00, 11391.50it/s]

87855


In [12]:
from copy import deepcopy
word_count2 = deepcopy(word_count)
word2toks2 = deepcopy(word2toks)

# Computing splits

In [13]:
word_count = deepcopy(word_count2)
word2toks = deepcopy(word2toks2)

In [14]:
for k, v in word_count.most_common(30):
    if len(word2toks[k]) > 1:
        print(word2toks[k])

['▁Та', 'в']
['▁ма', '̄', 'х', 'ум']
['▁по', 'рат']
['▁са', '̄', 'в']
['▁та', '̄', 'л']
['▁о', '̄', 'с']
['▁о', '̄', 'лыс']
['▁А', 'нь']
['▁о', '̄', 'лы']
['▁о', '̄', 'л', 'нэ']
['▁ма', '̄', 'н']
['▁ху', 'м']


In [15]:
steps = 100_000
min_count = 10 # 30
# default:   0 new tokens, 30 lenght, 0% new tokens
# 100 mindf: 6.6k new tokens, 22 length, 47% new tokens (of sentence length)
# 30 mindf:  20k new tokens, 20 length, 58% new tokens
# 10 mindf: 50K new tokens, 18.5 length, 64% new tokens
extra_vocab = []
extra_counts = []
extra_pairs = []

In [16]:
pairs_count = Counter()
pair2word = defaultdict(set)
for w, c in tqdm(word_count.items(), total=len(word_count)):
    enc = word2toks[w]
    for pair in zip(enc[:-1], enc[1:]):
        pairs_count[pair] += c
        pair2word[pair].add(w)

100%|██████████| 87855/87855 [00:00<00:00, 207781.07it/s]


In [17]:
%pip install heapdict -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
from heapdict import heapdict
hd = heapdict()
for w, c in pairs_count.items():
    hd[w] = -c

In [19]:
def replace_pair(old_tokens, pair, new_token):
    result = []
    prev = old_tokens[0]
    for tok in old_tokens[1:]:
        if (prev, tok) == pair:
            result.append(new_token)
            prev = None
        else:
            if prev is not None:
                result.append(prev)
            prev = tok
    if prev is not None:
        result.append(prev)
    return result

In [20]:
# rewrite with heapdict
for _ in trange(steps):
    #pair, c = pairs_count.most_common(1)[0]  # это самая времязатратная операция
    pair, c = hd.peekitem()
    c = -c

    if c < min_count:
        break
    new_token = ''.join(pair) # instead of BERT-like pair[0] + pair[1][2:]
    extra_vocab.append(pair)
    extra_counts.append(c)
    extra_pairs.append(pair)

    # update the vocabulary
    #new_id = len(id2ids)
    #tok2id[new_token] = new_id
    #id2ids.append(id2ids[tok2id[pair[0]]] + id2ids[tok2id[pair[1]]])

    # calculate the delta for the heap
    delta = Counter()
    for word in list(pair2word[pair]):
        # calculate old and new ways to tokenize the word
        old_toks = word2toks[word]
        # new_toks = " ".join(old_toks).replace(' '.join(pair), new_token).split(" ")
        new_toks = replace_pair(old_toks, pair, new_token)
        word2toks[word] = new_toks
        wc = word_count[word]
        # update the index concerning the tokens of the word
        for old_pair in zip(old_toks[:-1], old_toks[1:]):
            #pairs_count[old_pair] -= wc
            delta[old_pair] -= wc
            if word in pair2word[old_pair]:
                pair2word[old_pair].remove(word)
        for new_pair in zip(new_toks[:-1], new_toks[1:]):
            # pairs_count[new_pair] += wc
            delta[new_pair] += wc
            pair2word[new_pair].add(word)
    # update the heap
    for a_pair, a_delta in delta.items():
        if a_delta == 0:
            continue
        if a_pair not in hd:
            hd[a_pair] = 0
        hd[a_pair] -= a_delta

 11%|█         | 11105/100000 [00:06<00:48, 1816.07it/s]


In [21]:
len(extra_pairs)

11105

In [22]:
extra_pairs[:10]

[('▁о', '̄'),
 ('▁ма', '̄'),
 ('̄', 'т'),
 ('̄', 'н'),
 ('ӈ', 'к'),
 ('▁ва', '̄'),
 ('̄', 'в'),
 ('▁а', '̄'),
 ('▁та', '̄'),
 ('ӈк', 'ве')]

In [23]:
extra_pairs[-20:]

[('▁тот', 'ыла'),
 ('▁Иосиф', 'н'),
 ('▁пыг', 'на'),
 ('▁Сак', 'въ'),
 ('▁лӯпта', 'м'),
 ('ха', '̄тнэ'),
 ('ӯнлан', 'ты'),
 ('пы̄г', 'кве'),
 ('▁Та', 'сир'),
 ('▁па', 'саныл'),
 ('щи', 'п'),
 ('▁Ху', 'рум'),
 ('глас', 'ын'),
 ('▁то̄', 'варт'),
 ('ё̄ӈ', 'хатах'),
 ('▁хӯ', 'рый'),
 ('▁пумы', '̄т'),
 ('ат', 'ве̄сыт'),
 ('▁хар', 'тэ̄гум'),
 ('йт', 'асыт')]

In [24]:
tokenizer.vocab_size

250054

In [25]:
tokenizer.save_pretrained('old_tokenizer')

('old_tokenizer/tokenizer_config.json',
 'old_tokenizer/special_tokens_map.json',
 'old_tokenizer/sentencepiece.bpe.model',
 'old_tokenizer/added_tokens.json')

In [26]:
import sentencepiece_model_pb2 as model
m = model.ModelProto()
m.ParseFromString(open("old_tokenizer/sentencepiece.bpe.model", "rb").read())

5069051

In [27]:
scores = [p.score for p in m.pieces]
min_score = min(scores)
epsilon = 1e-4

In [28]:
tokenizer.tokenize('приввет')

['▁прив', 'вет']

In [29]:
type(m.pieces[37764-1].piece)

str

In [30]:
for i, pair in enumerate(extra_vocab):
    new_token = model.ModelProto().SentencePiece()
    new_token.piece = ''.join(pair)
    new_token.score = min_score - epsilon * (i+1)
    m.pieces.append(new_token)

In [31]:
with open("old_tokenizer/sentencepiece.bpe.model", 'wb') as f:
    f.write(m.SerializeToString())

In [32]:
new_tokenizer = MBart50Tokenizer.from_pretrained(
    "old_tokenizer/sentencepiece.bpe.model",
    additional_special_tokens = tokenizer.additional_special_tokens
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2165: FutureWarning: Calling MBart50Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [33]:
new_tokenizer.vocab_size

261159

In [34]:
from transformers.models.mbart50.tokenization_mbart50 import FAIRSEQ_LANGUAGE_CODES

In [35]:
self = new_tokenizer

In [36]:
self.lang_code_to_id = {
    code: self.sp_model_size + i + self.fairseq_offset for i, code in enumerate(FAIRSEQ_LANGUAGE_CODES + ['mans_XX'])
}
self.id_to_lang_code = {v: k for k, v in self.lang_code_to_id.items()}
self.fairseq_tokens_to_ids["<mask>"] = len(self.sp_model) + len(self.lang_code_to_id) + self.fairseq_offset

self.fairseq_tokens_to_ids.update(self.lang_code_to_id)
self.fairseq_ids_to_tokens = {v: k for k, v in self.fairseq_tokens_to_ids.items()}



In [37]:
new_tokenizer.additional_special_tokens.append('mans_XX')

In [38]:
new_tokenizer.vocab_size

261160

In [39]:
len(new_tokenizer)

261159

In [40]:
text = random.choice(all_sentences)
print(text)

Тамле э̄рнэ тэ̄ла ва̄руӈкве этнографыг о̄лнэ ханты нэ̄ Татьяна Александровна Молданова ос искусствовед нэ̄ Наталья Николаевна Фёдорова номылматсы̄г.


In [41]:
print(tokenizer.tokenize(text))
print(new_tokenizer.tokenize(text))

['▁Там', 'ле', '▁э', '̄', 'р', 'нэ', '▁тэ', '̄', 'ла', '▁ва', '̄', 'ру', 'ӈ', 'к', 'ве', '▁этно', 'граф', 'ыг', '▁о', '̄', 'л', 'нэ', '▁хан', 'ты', '▁', 'нэ', '̄', '▁Татьяна', '▁Александр', 'овна', '▁Мол', 'дан', 'ова', '▁ос', '▁искусство', 'вед', '▁', 'нэ', '̄', '▁Наталья', '▁Никола', 'ев', 'на', '▁Ф', 'ё', 'дор', 'ова', '▁но', 'мыл', 'мат', 'сы', '̄', 'г', '.']
['▁Тамле', '▁э̄рнэ', '▁тэ̄ла', '▁ва̄руӈкве', '▁этнограф', 'ыг', '▁о̄лнэ', '▁ханты', '▁', 'нэ̄', '▁Татьяна', '▁Александровна', '▁Молданова', '▁ос', '▁искусство', 'вед', '▁', 'нэ̄', '▁Наталья', '▁Николаевна', '▁Фёдор', 'ова', '▁номылмат', 'сы̄г', '.']


In [42]:
sample = all_sentences[:10000]

In [43]:
import pandas as pd
pd.DataFrame({
    'old': [len(tokenizer.tokenize(text)) for text in sample],
    'new': [len(new_tokenizer.tokenize(text)) for text in sample]
}).describe()

,old,new
count,10000.000000,10000.00000
mean,22.497600,11.96860
std,14.293909,7.81571
min,3.000000,2.00000
25%,13.000000,7.00000
50%,18.000000,10.00000
75%,28.000000,15.00000
max,154.000000,90.00000


In [44]:
new_tot_len, tot_len = 0, 0
for text in sample:
    for tok in new_tokenizer.tokenize(text):
        s = len(tok)
        tot_len += s
        if new_tokenizer.convert_tokens_to_ids(tok) > len(tokenizer.sp_model):
            new_tot_len += s
print(new_tot_len / tot_len)

0.6875596496313211


In [45]:
len(new_tokenizer)

261159

In [46]:
len(m.pieces)

261105

In [47]:
# model.resize_token_embeddings(len(tokenizer))
# todo: move language id embeddings by the distance.

# Calculating tokens alignment

In [48]:
import pandas as pd
data = pd.read_csv('overall_80K.csv')
print(data.shape[0])

81146


In [49]:
from collections import defaultdict, Counter
token_priors = Counter()
token_to_others = defaultdict(Counter)

In [50]:
for mans, ru in tqdm(zip(data["target"].values.tolist(), data["source"].values.tolist()), total=len(data["target"].values.tolist())):
    mans_toks = new_tokenizer.convert_tokens_to_ids(new_tokenizer.tokenize(mans))
    ru_toks = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ru))
    token_priors.update(ru_toks)
    for mans_tok in mans_toks:
        token_to_others[mans_tok].update(ru_toks)

100%|██████████| 81146/81146 [00:16<00:00, 4911.79it/s]


In [51]:
mans_tok = len(new_tokenizer)

In [52]:
import numpy as np

def get_ru_toks(mans_tok):
    ru_toks = []
    ru_weights = []
    for t, w in token_to_others[mans_tok].items():
        ru_toks.append(t)
        ru_weights.append(w**2 / token_priors[t])
    ru_weights = np.array(ru_weights)
    ru_weights = ru_weights / (sum(ru_weights) + 1e-4)
    return ru_weights, ru_toks

In [53]:
mans_tok

261159

In [54]:
sorted(zip(*get_ru_toks(mans_tok)), reverse=True)

[]

In [55]:
tokenizer.convert_ids_to_tokens([62994, 17921, 117229, 188048, 25982])

['▁первую', '▁справ', '▁карты', 'ведению', '▁ат']

# Updating the model embeddings

In [56]:
len(new_tokenizer)

261159

In [57]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [58]:
model.model.shared

MBartScaledWordEmbedding(250054, 1024, padding_idx=1)

In [60]:
model.resize_token_embeddings(len(new_tokenizer) + 1)

#assert model embds shape == len(new_tokenizer)

MBartScaledWordEmbedding(261160, 1024, padding_idx=1)

Move added tokens to the end

In [61]:
n_extra = len(extra_vocab)
print(n_extra)
old_vocab_size = len(tokenizer.sp_model) + 1

11105


In [62]:
for old_token_id in range(old_vocab_size, len(tokenizer)):
    old_token = tokenizer.convert_ids_to_tokens(old_token_id)
    new_token_id = new_tokenizer.convert_tokens_to_ids(old_token)

    print(old_token_id, old_token, new_token_id)
    # model.model.shared.weight.data[i + n_extra] = model.model.shared.weight.data[i]
    model.model.shared.weight.data[new_token_id] = model.model.shared.weight.data[old_token_id]

250001 ar_AR 261106
250002 cs_CZ 261107
250003 de_DE 261108
250004 en_XX 261109
250005 es_XX 261110
250006 et_EE 261111
250007 fi_FI 261112
250008 fr_XX 261113
250009 gu_IN 261114
250010 hi_IN 261115
250011 it_IT 261116
250012 ja_XX 261117
250013 kk_KZ 261118
250014 ko_KR 261119
250015 lt_LT 261120
250016 lv_LV 261121
250017 my_MM 261122
250018 ne_NP 261123
250019 nl_XX 261124
250020 ro_RO 261125
250021 ru_RU 261126
250022 si_LK 261127
250023 tr_TR 261128
250024 vi_VN 261129
250025 zh_CN 261130
250026 af_ZA 261131
250027 az_AZ 261132
250028 bn_IN 261133
250029 fa_IR 261134
250030 he_IL 261135
250031 hr_HR 261136
250032 id_ID 261137
250033 ka_GE 261138
250034 km_KH 261139
250035 mk_MK 261140
250036 ml_IN 261141
250037 mn_MN 261142
250038 mr_IN 261143
250039 pl_PL 261144
250040 ps_AF 261145
250041 pt_XX 261146
250042 sv_SE 261147
250043 sw_KE 261148
250044 ta_IN 261149
250045 te_IN 261150
250046 th_TH 261151
250047 tl_XX 261152
250048 uk_UA 261153
250049 ur_PK 261154
250050 xh_ZA 261155


Because we have added one more language, its id must be computed separately, e.g. as an average of related languages.

In [63]:
model.model.shared.weight.data[new_tokenizer.convert_tokens_to_ids('mans_XX')] = (
    model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('fi_FI')] * 0.25
    + model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('et_EE')] * 0.25
    + model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('ru_RU')] * 0.25
    + model.model.shared.weight.data[tokenizer.convert_tokens_to_ids('sv_SE')] * 0.25
)

Compute embeddings for newly added tokens

In [64]:
for i in trange(n_extra):
    mans_tok = i + old_vocab_size
    ru_weights, ru_toks = get_ru_toks(mans_tok)
    if len(ru_toks) > 0:
        new_embedding = (model.model.shared.weight.data[ru_toks].T * ru_weights).sum(1)
        model.model.shared.weight.data[mans_tok] = new_embedding

100%|██████████| 11105/11105 [00:12<00:00, 861.85it/s] 


In [65]:
# !mkdir /gd/MyDrive/models/myv

In [66]:
new_model_path = './mbart-large-51-mans-raw'

Check that the model is still able to translate texts.  

In [67]:
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"

# translate Hindi to French
new_tokenizer.src_lang = "hi_IN"
encoded_hi = new_tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["fr_XX"],
    max_length=30,
    num_beams=1
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


["Le chef de la mission de l 'ONU a déclaré qu 'il n' y a pas de solution militaire en Syria."]

Translation to Russian has broken, because some new myv tokens are very Russian-like, and they interere.

In [68]:
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["ru_RU"],
    max_length=30,
    num_beams=5,
    repetition_penalty=30.0,
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

["Ща The head of the United Nations says that there's no military solution in Syria. < s > Relatedиян: http://"]

For some reason, even with new tokens prohibited, translation to Russian is poor.

In [69]:
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["ru_RU"],
    max_length=30,
    num_beams=5,
    repetition_penalty=30.0,
    bad_words_ids=[[t] for t in range(n_extra, n_extra+old_vocab_size)]
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire dans la Syrie."

["<mask>. < s > The head of the UN said there's no armed settler in Siria; [c]"]

In [70]:
new_tokenizer.src_lang = "mans_XX"
print(all_sentences[0])
encoded_hi = new_tokenizer(all_sentences[0], return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=new_tokenizer.lang_code_to_id["ru_RU"],
    max_length=30,
    num_beams=5,
    repetition_penalty=30.0
)
new_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

Та пыгрисит маим вармаль э̄рнэ поратэт ат вермгыг варункв


['Гермоген maим сяр округеat вермг мулы вар смена ( ех Сос румасаге)  Хумнитэ верме Maim Аньнья утэ']

In [71]:
old_len = len(new_tokenizer)
new_tokenizer.lang_code_to_id['mans_XX'] = old_len-1
new_tokenizer.id_to_lang_code[old_len-1] = 'mans_XX'
new_tokenizer.fairseq_tokens_to_ids["<mask>"] = len(new_tokenizer.sp_model) + len(new_tokenizer.lang_code_to_id) + new_tokenizer.fairseq_offset

new_tokenizer.fairseq_tokens_to_ids.update(new_tokenizer.lang_code_to_id)
new_tokenizer.fairseq_ids_to_tokens = {v: k for k, v in new_tokenizer.fairseq_tokens_to_ids.items()}
if 'mans_XX' not in tokenizer._additional_special_tokens:
    new_tokenizer._additional_special_tokens.append('mans_XX')



In [72]:
new_tokenizer.fairseq_tokens_to_ids["<mask>"]

261159

In [73]:
new_tokenizer.lang_code_to_id["mans_XX"], new_tokenizer.lang_code_to_id["ru_RU"]

(261158, 261126)

In [74]:
print("BEFORE", new_tokenizer.cur_lang_code_id, new_tokenizer.src_lang, new_tokenizer.lang_code_to_id[new_tokenizer.src_lang])


new_tokenizer.src_lang = "ru_RU"

print("AFTER", new_tokenizer.cur_lang_code_id, new_tokenizer.src_lang, new_tokenizer.lang_code_to_id[new_tokenizer.src_lang])

BEFORE 261158 mans_XX 261158
AFTER 261126 ru_RU 261126


In [75]:
model.save_pretrained(new_model_path)
new_tokenizer.save_pretrained(new_model_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


('./mbart-large-51-mans-raw/tokenizer_config.json',
 './mbart-large-51-mans-raw/special_tokens_map.json',
 './mbart-large-51-mans-raw/sentencepiece.bpe.model',
 './mbart-large-51-mans-raw/added_tokens.json')